In [79]:
import pandas as pd
import re
import operator
import calendar
import urllib.request, urllib.error, urllib.parse
from datetime import datetime
from bs4 import BeautifulSoup
# Get Tourneys
fileName=('BUWomensRecordBook.txt')
tourneys=[]
teamSet=set()
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    tourneyDict={}
    for i in rows:
        row=i.split('  ')
        if(len(row)==2):
            tourneyDict[row[0]]=row[1]
            #print(row)
            

In [82]:
# Get Games
fileName=('BUWomensRecordBook.txt')
gameList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:      
        if(len(i)==7):
            season=i 
        coachSearch=re.search("COACH: (.*)",i)
        if coachSearch != None:
            coach=coachSearch.group(1)
        captSearch=re.search("CAPTAIN.?: (.*)",i)
        if captSearch != None:
            capt=captSearch.group(1) 
            #print(capt)
        confSearch=re.search("(NEIHL|ECAC|HOCKEY EAST):",i)
        if confSearch != None:
            conf=confSearch.group(1)
            if(conf=='HOCKEY EAST'):
                conf='Hockey East'
        if(re.search('\*',i)!=None):
            gameType='Conference'
        else:
            gameType='Non-Conference'
        i=i.replace("* ",'')
        note=''
        if('†' in i):
            note='Loss by forfeit (ineligible player)'
            i=i.replace('†','')
        if('+' in i):
            note='Win by forfeit (opponent left ice – score was 5-1, BU)'
            i=i.replace('+','')
        if('‡' in i):
            note='Win by forfeit (ineligible player)'
            i=i.replace('‡','')
        game=re.search(r"(\d*\/\d*) (\w) (?:\((.?ot)\))? ?(.*) (\w) (#\d*)? ?(\S*|\S* \S*|\S* \S* \S*) ?(\(.*\))? (\d*-\d*)",i)
        if game==None:
            continue
        #print(game.groups())
        gameDict={'date':game.group(1),
                 'result':game.group(2),
                 'ot':game.group(3),
                 'arena':game.group(4),
                 'opponent':game.group(7),
                 'gameType':gameType,
                 'tourney': game.group(8),
                 'scoreline':game.group(9),
                 'location':game.group(5),
                 'rank':game.group(6),
                 'coach':coach,
                 'captain':capt,
                 'conference':conf,
                 'season':season,
                 'note':note}
        if(gameDict['gameType']==None):
            gameDict['gameType']='Non-Conference'
        if(gameDict['tourney']=='(ex)' or gameDict['result']=='E'):
            gameDict['gameType']='Exhibition'
            gameDict['result'] = 'E'
            gameDict['tourney'] = None

        if(gameDict['tourney']!=None):
            #print(gameDict)
            gameDict['tourney']=tourneyDict[gameDict['tourney'].replace('(','').replace(')','')]
        if((gameDict['tourney'] == gameDict['conference'] +" "+ 'Tournament') or (gameDict['tourney'] == 'NCAA Tournament')):
            gameDict['seasonType'] = 'Playoffs'
        else:
            gameDict['seasonType'] = 'Regular Season'
        if(gameDict['location']=='H'):
            gameDict['location']='Home'
        elif(gameDict['location']=='A'):
            gameDict['location']='Away'
        elif(gameDict['location']=='N'):
            gameDict['location']='Neutral'
        gameDict['month'],gameDict['day']=gameDict['date'].split('/')
        gameDict['month']=int(gameDict['month'])
        gameDict['day']=int(gameDict['day'])
        if(gameDict['month'] >=9):
            gameDict['date']+="/" + gameDict['season'][:4]
            gameDict['year']=int(gameDict['season'][:4])
        elif(gameDict['month'] <= 4):
            gameDict['date']+= "/" + str(int(gameDict['season'][:4])+1)
            gameDict['year']=int(gameDict['season'][:4])+1
        gameDict['BUScore'],gameDict['OppoScore']=int(gameDict['scoreline'].split('-')[0]),int(gameDict['scoreline'].split('-')[1])
        gameDict['GD']=abs(gameDict['BUScore']-gameDict['OppoScore'])
        gameDict['date']=pd.Timestamp(gameDict['date'])
        gameDict['dow']=gameDict['date'].weekday()
        gameList.append(gameDict)
f.close()
dfWomensGames=pd.DataFrame(gameList)

In [83]:
dfWomensGames

,date,result,ot,arena,opponent,gameType,tourney,scoreline,location,rank,...,season,note,seasonType,month,day,year,BUScore,OppoScore,GD,dow
0,2005-09-30,E,None,Walter Brown Arena,Bluewater Hawks,Exhibition,None,3-5,Home,None,...,2005-06,,Regular Season,9,30,2005,3,5,2,4
1,2005-10-07,L,None,Matthews Arena,Northeastern,Conference,None,3-4,Away,None,...,2005-06,,Regular Season,10,7,2005,3,4,1,4
2,2005-10-11,T,ot,Walter Brown Arena,Quinnipiac,Non-Conference,None,2-2,Home,None,...,2005-06,,Regular Season,10,11,2005,2,2,0,1
3,2005-10-14,W,None,Walter Brown Arena,Union,Non-Conference,None,4-1,Home,None,...,2005-06,,Regular Season,10,14,2005,4,1,3,4
4,2005-10-15,W,None,Walter Brown Arena,Union,Non-Conference,None,6-2,Home,None,...,2005-06,,Regular Season,10,15,2005,6,2,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,2022-02-12,L,None,Gutterson,Vermont,Conference,None,2-5,Away,None,...,2021-22,,Regular Season,2,12,2022,2,5,3,5
608,2022-02-15,W,ot,Walter Brown Arena,Maine,Conference,None,3-2,Home,None,...,2021-22,,Regular Season,2,15,2022,3,2,1,1
609,2022-02-18,T,None,(sol) Schneider Arena,Providence,Conference,None,1-1,Away,None,...,2021-22,,Regular Season,2,18,2022,1,1,0,4
610,2022-02-19,L,ot,Walter Brown Arena,Providence,Conference,None,1-2,Home,None,...,2021-22,,Regular Season,2,19,2022,1,2,1,5
